** This file is for cleaning the tweets and prepare it for tmporal analysis and sentiment analysis using ML**

In [1]:
# To use cleaning process if needed
from utils import *
from config_file import DOWNLOAD_ROOT, DATASETS_PATH, DOWNLOAD_URL, TWEETS_PATH

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import string
import unicodedata
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
import re
from pyspark.conf import SparkConf
from pyspark.sql import functions as F
import string
import csv
from pyspark.sql.types import StringType, TimestampType
from pyspark.sql.types import ArrayType

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
PROJECT_ID = "happiness_over_countries"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", PROJECT_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
spark = SparkSession.builder\
    .master("local[*]")\
    .appName("nlp")\
    .config("spark.executor.memory", "32g")\
    .config("spark.driver.memory", "32g")\
    .config("spark.memory.offHeap.enabled",True) \
    .config("spark.memory.offHeap.size","16g")\
    .config("spark.debug.maxToStringFields","200")\
    .getOrCreate()

In [ ]:
if __name__ == '__main__':
    extract_tar_file(tgz_name = "datasets.tar.gz")
    for i in range(1,36):
        filename = str(i)+'.csv'
        tweets = load_data(TWEETS_PATH, filename)
        tweets = tweets.selectExpr("_c0 as date", "_c1 as text")
        tweets = tweets.withColumn("text", udfhttps(tweets["text"]))
        tweets = tweets.withColumn("text", udfNormalizeData(tweets["text"]))
        tweets = tweets.withColumn("text", udfDecoding(tweets["text"]))
        tweets= lower_words(tweets)
        tweets= regexTokenizer_StopWordsRemover(tweets, 'text', 'words_','words')
        tweets = remove_emoji(tweets, "words","words_")
        #tweets.show(2)
        if i == 1:
            tweets = avg_happiness_of_words(tweets, 'words')
            tweets_final = tweets

        else:
            tweets = avg_happiness_of_words(tweets, 'words')
            tweets_final = tweets_final.union(tweets)
            
        tweets_final.write.json(os.path.join(DATASETS_PATH,'sentiment_analysis'))